# Part 1

You land in an airport surrounded by dense forest. As you walk to your high-speed train, the Elves at the Mythical Information Bureau contact you again. They think their satellite has collected an image of a sea monster! Unfortunately, the connection to the satellite is having problems, and many of the messages sent back from the satellite have been corrupted.

They sent you a list of the rules valid messages should obey and a list of received messages they've collected so far (your puzzle input).

The rules for valid messages (the top part of your puzzle input) are numbered and build upon each other. For example:
```
0: 1 2
1: "a"
2: 1 3 | 3 1
3: "b"
```
Some rules, like `3: "b"`, simply match a single character (in this case, `b`).

The remaining rules list the sub-rules that must be followed; for example, the rule `0: 1 2` means that to match rule 0, the text being checked must match rule 1, and the text after the part that matched rule 1 must then match rule 2.

Some of the rules have multiple lists of sub-rules separated by a pipe (`|`). This means that at least one list of sub-rules must match. (The ones that match might be different each time the rule is encountered.) For example, the rule `2: 1 3 | 3 1` means that to match rule 2, the text being checked must match rule 1 followed by rule 3 or it must match rule 3 followed by rule 1.

Fortunately, there are no loops in the rules, so the list of possible matches will be finite. Since rule 1 matches `a` and rule 3 matches `b`, rule 2 matches either `ab` or `ba`. Therefore, rule 0 matches `aab` or `aba`.

Here's a more interesting example:
```
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"
```

Here, because rule 4 matches `a` and rule 5 matches `b`, rule 2 matches two letters that are the same (`aa` or `bb`), and rule 3 matches two letters that are different (`ab` or `ba`).

Since rule 1 matches rules 2 and 3 once each in either order, it must match two pairs of letters, one pair with matching letters and one pair with different letters. This leaves eight possibilities: `aaab`, `aaba`, `bbab`, `bbba`, `abaa`, `abbb`, `baaa`, or `babb`.

Rule 0, therefore, matches `a` (rule 4), then any of the eight options from rule 1, then `b` (rule 5): `aaaabb`, `aaabab`, `abbabb`, `abbbab`, `aabaab`, `aabbbb`, `abaaab`, or `ababbb`.

The received messages (the bottom part of your puzzle input) need to be checked against the rules so you can determine which are valid and which are corrupted. Including the rules and the messages together, this might look like:
```
0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb
```
Your goal is to determine the number of messages that completely match rule 0. In the above example, `ababbb` and `abbbab` match, but `bababa`, `aaabbb`, and `aaaabbb` do not, producing the answer 2. The whole message must match all of rule 0; there can't be extra unmatched characters in the message. (For example, `aaaabbb` might appear to match rule 0 above, but it has an extra unmatched b on the end.)

How many messages completely match rule 0?

In [1]:
# Possibly it's easier to recursively use string.replace on the rules to get a regexp and use
# python's built-in regexp functionality. But it seems more fun to implement an own minimalistic
# regexp compiler.


from abc import ABC, abstractmethod
from collections import namedtuple


Match = namedtuple('Match', ['start', 'end', 'match'])


class Pattern(ABC):
    @abstractmethod
    def match(self, string):
        pass

    
class StartsWith(Pattern):
    def __init__(self, pattern):
        self._p = pattern
    
    def match(self, string):
        start = string.find(self._p)
        if start == 0:
            end = len(self._p)
            m = string[:end]
            return Match(0, end, m)
    
    def __str__(self):
        return str(self._p)


class Or(Pattern):
    def __init__(self, pattern1, pattern2):
        self._p1 = pattern1
        self._p2 = pattern2
    
    def match(self, string):
        m = self._p1.match(string)
        if m is not None:
            return m
        else:
            return self._p2.match(string)
    
    def __str__(self):
        return '(%s | %s)' % (self._p1, self._p2)
    

class Concatenate(Pattern):
    def __init__(self, *patterns):
        self._patterns = patterns
        
    def match(self, string):
        m = self._patterns[0].match(string)
        if m is None:
            return
        start = m.start
        end = m.end
        substr = string[m.end:]
        for p in self._patterns[1:]:
            m = p.match(substr)
            if m is None or m.start != 0:
                return
            substr = substr[m.end:]
            end += m.end
        m = string[start:end]
        return Match(start, end, m)
    
    def __str__(self):
        return ''.join([str(p) for p in self._patterns])

        
class InputParser:
    def __init__(self):
        self._mode = 'rules'
        self._inp = {'rules': {}, 'messages': []}
    
    def parse(self, line):
        line = line.rstrip()
        if line == '':
            self._mode = 'messages'
        elif self._mode == 'rules':
            rule_id, rule = line.split(': ')
            self._inp['rules'][rule_id] = rule
        elif self._mode == 'messages':
            self._inp['messages'].append(line)
        else:
            raise Exception('Failed to parse line %s in mode %s.' % (line, self._mode))
    
    def result(self):
        return self._inp
            

def get_input(parser=None):
    if parser == None:
        parser = InputParser()
    with open('inputs/19') as f:
        for line in f.readlines():
            parser.parse(line)
    return parser.result()


def compile_rules(rules, rule_num=0):
    _ALREADY_COMPILED = {}
    return _compile_rule(str(rule_num), rules, _ALREADY_COMPILED)


def _compile_rule(rule_num, rules, already_compiled):
    compiled = already_compiled.get(rule_num)
    if compiled is not None:
        return compiled
    rule = rules[rule_num]
    if rule.startswith('"') and rule.endswith('"') and len(rule) == 3:  # '"<letter>"', e.g. '"b"'
        compiled = StartsWith(rule[1])
    else:
        or_operands = [Concatenate(*[_compile_rule(i, rules, already_compiled) for i in o.split()]) for o in rule.split(' | ')]
        if len(or_operands) == 1:
            compiled = or_operands[0]
        else:
            compiled = Or(*or_operands)
    already_compiled[rule_num] = compiled
    return compiled


def is_valid_message(rule, msg):
    m = rule.match(msg)
    if m is not None and m.end == len(msg):
        return True
    return False


def count_valid_messages(rules, messages):
    rule = compile_rules(rules)
    return sum(is_valid_message(rule, msg) for msg in messages)


# Test
rules = {
    '0': '4 1 5',
    '1': '2 3 | 3 2',
    '2': '4 4 | 5 5',
    '3': '4 5 | 5 4',
    '4': '"a"',
    '5': '"b"'
}
rule = compile_rules(rules)
good1 = rule.match('ababbb')
good2 = rule.match('abbbab')
bad1 = rule.match('bababa')
bad2 = rule.match('aaabbb')
bad3 = rule.match('aaabbb')
assert good1, good1
assert good2, good2
assert not bad1, bad1
assert not bad2, bad2
assert not bad3, bad3
assert count_valid_messages(rules, ['ababbb', 'abbbab', 'bababa', 'aaabbb', 'aaabbb']) == 2


def result1():
    inp = get_input()
    rules = inp['rules']
    messages = inp['messages']
    rule = compile_rules(rules)
    return count_valid_messages(rules, messages)


result1()

198

# Part 2

As you look over the list of messages, you realize your matching rules aren't quite right. To fix them, completely replace rules 8: 42 and 11: 42 31 with the following:
```
8: 42 | 42 8
11: 42 31 | 42 11 31
```
This small change has a big impact: now, the rules do contain loops, and the list of messages they could hypothetically match is infinite. You'll need to determine how these changes affect which messages are valid.

Fortunately, many of the rules are unaffected by this change; it might help to start by looking at which rules always match the same set of values and how those rules (especially rules 42 and 31) are used by the new versions of rules 8 and 11.

(Remember, you only need to handle the rules you have; building a solution that could handle any hypothetical combination of rules would be significantly more difficult.)

For example:
```
42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
```

Without updating rules 8 and 11, these rules only match three messages: `bbabbbbaabaabba`, `ababaaaaaabaaab` and `ababaaaaabbbaba`.

However, after updating rules 8 and 11, a total of 12 messages match:
```
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
```
After updating rules 8 and 11, how many messages completely match rule 0?



In [2]:
# For the second part we need a smarter regexp engine that one above so instead of using it let's just
# convert the rules into regexp and use python's own regexp engine.

from string import ascii_lowercase
import re


def convert_rules_to_regexp(rules, rule_num=0):
    rule_num = str(rule_num)
    rule = rules[rule_num]
    parts = rule.split()
    regexp = ''
    open_parentheses = 0
    for p in parts:
        m = re.match(r'"(\w)"', p)
        if m is not None:
            regexp += m.group(1)
        elif p == '|':
            if open_parentheses > 0:
                regexp += ')'
                open_parentheses -= 1
            regexp = '(%s|' % regexp
            open_parentheses += 1
        else:
            regexp += convert_rules_to_regexp(rules, p)
    regexp += (')' * open_parentheses)
    return regexp


inp = get_input()
rules = inp['rules']
messages = inp['messages']
regexp = convert_rules_to_regexp(rules)

def is_valid_message(regexp, msg):
    m = re.match(regexp, msg)
    if m is not None and m.span()[1] == len(msg):
        return True
    return False


def count_valid_messages(rules, messages):
    regexp = convert_rules_to_regexp(rules)
    return sum(is_valid_message(regexp, msg) for msg in messages)


def test():
    inp = get_input()
    rules = inp['rules']
    messages = inp['messages']
    cnt = count_valid_messages(rules, messages)
    assert cnt == result1()


test()


def result2():
    inp = get_input()
    rules = inp['rules']
    messages = inp['messages']
    versions = ['', *list(ascii_lowercase)]
    for x, y in zip(versions[:-1], versions[1:]):
        rules['8' + x] = '42 | 42 8%s' % y
        rules['11' + x] = '42 31 | 42 11%s 31' % y
    rules['8' + versions[-1]] = '42 | 42 42'
    rules['11' + versions[-1]] = '42 31 | 42 42 31 31'
    return count_valid_messages(rules, messages)


result2()

372